<h3>Import required classes.</h3>

In [1]:
import json

from classes.scorecard import Scorecard
from classes.affordability import Affordability
from classes.pricing import Pricing

<h3>Read in the loan applicant's details.</h3>

In [2]:
with open('resources/pricingtest.json') as json_data:
    pricingtest = json.load(json_data)
    json_data.close()
pricingtest.extend(pricingtest)
print(json.dumps(pricingtest, indent=4))

[
    {
        "monthly_income": 5000,
        "mortgage_or_rent": 200,
        "monthly_credit_commitments": 0,
        "employment_status": "full_time",
        "no_of_adults": 1,
        "no_of_dependants": 0,
        "Age": 67,
        "Sex": "male",
        "Job": 2,
        "Housing": "own",
        "Saving accounts": null,
        "Checking account": "little",
        "Purpose": "radio/TV"
    },
    {
        "monthly_income": 5000,
        "mortgage_or_rent": 200,
        "monthly_credit_commitments": 0,
        "employment_status": "full_time",
        "no_of_adults": 1,
        "no_of_dependants": 0,
        "Age": 67,
        "Sex": "male",
        "Job": 2,
        "Housing": "own",
        "Saving accounts": null,
        "Checking account": "little",
        "Purpose": "radio/TV"
    }
]


<h3>Associate with a pricing matrix of interest rates, loan terms and values.</h3>

In [3]:
pricing = Pricing(pricingtest)
pricing._df.head()

,Age,Checking account,Housing,Job,Purpose,Saving accounts,Sex,employment_status,monthly_credit_commitments,monthly_income,mortgage_or_rent,no_of_adults,no_of_dependants,Application identifier,Interest rate,Duration,Credit amount,Monthly payment
0,67,little,own,2,radio/TV,None,male,full_time,0,5000,200,1,0,0,0.05,4,250,63.152395
1,67,little,own,2,radio/TV,None,male,full_time,0,5000,200,1,0,0,0.05,4,500,126.304790
2,67,little,own,2,radio/TV,None,male,full_time,0,5000,200,1,0,0,0.05,4,1000,252.609581
3,67,little,own,2,radio/TV,None,male,full_time,0,5000,200,1,0,0,0.05,4,2500,631.523952
4,67,little,own,2,radio/TV,None,male,full_time,0,5000,200,1,0,0,0.05,4,5000,1263.047904


<h3>Instantiate scorecard and use to estimate credit risk.</h3>

In [4]:
scorecard = Scorecard()
pricing.calculate_credit_risk(scorecard)
pricing._df.head()

,Age,Checking account,Housing,Job,Purpose,Saving accounts,Sex,employment_status,monthly_credit_commitments,monthly_income,...,no_of_dependants,Application identifier,Interest rate,Duration,Credit amount,Monthly payment,pBad,pGood,Total cost,Profit
0,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0,0,0.05,4,250,63.152395,0.114854,0.885146,252.609581,-26.403671
1,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0,0,0.05,4,500,126.304790,0.115552,0.884448,505.219161,-53.159892
2,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0,0,0.05,4,1000,252.609581,0.116959,0.883041,1010.438323,-107.741160
3,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0,0,0.05,4,2500,631.523952,0.121268,0.878732,2526.095807,-280.239040
4,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0,0,0.05,4,5000,1263.047904,0.128754,0.871246,5052.191614,-598.300716


<h3>Similarly, instantiate affordability risk assessment and annotate the table</h3>

In [5]:
affordability = Affordability()
pricing.calculate_affordability(affordability)
pricing._df.head()

,Age,Checking account,Housing,Job,Purpose,Saving accounts,Sex,employment_status,monthly_credit_commitments,monthly_income,...,Interest rate,Duration,Credit amount,Monthly payment,pBad,pGood,Total cost,Profit,expenditure,affordability
0,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0.05,4,250,63.152395,0.114854,0.885146,252.609581,-26.403671,1727.625,1636.1875
1,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0.05,4,500,126.304790,0.115552,0.884448,505.219161,-53.159892,1727.625,1636.1875
2,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0.05,4,1000,252.609581,0.116959,0.883041,1010.438323,-107.741160,1727.625,1636.1875
3,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0.05,4,2500,631.523952,0.121268,0.878732,2526.095807,-280.239040,1727.625,1636.1875
4,67,little,own,2,radio/TV,None,male,full_time,0,5000,...,0.05,4,5000,1263.047904,0.128754,0.871246,5052.191614,-598.300716,1727.625,1636.1875


<h3>Finally, choose suitable profit level and filter results.</h3>

In [6]:
pricing.get_suitable_products(-25)

Interest rate  Monthly payment
Application identifier Duration Credit amount                                
0                      4        250                     0.10        63.807486
                       8        250                     0.10        32.433220
                       12       250                     0.10        21.978972
                       24       250                     0.15        12.121662
1                      4        250                     0.10        63.807486
                       8        250                     0.10        32.433220
                       12       250                     0.10        21.978972
                       24       250                     0.15        12.121662